In [3]:
import pandas as pd
import numpy as np
import sklearn
from importlib import util

In [4]:
companies = pd.read_parquet('companies.parquet', engine='fastparquet')
deals = pd.read_parquet('deals.parquet', engine='fastparquet')
print('Loaded companies and deals; join is handled by EDA (data_clean.ipynb)')

Loaded companies and deals; join is handled by EDA (data_clean.ipynb)


In [5]:
print('Joining companies onto deals (left join on CompanyID)')

_deals = deals.copy()
_companies = companies.copy()

deals_joined = _deals.merge(_companies, on='CompanyID', how='left')
matched = deals_joined['RowID_y'].notna().sum()

print(f'Before shape: {_deals.shape}, After shape: {deals_joined.shape}, Matched company rows: {matched}')

# write joined parquet for reuse by EDA and other scripts
joined_path = 'deals_joined.parquet'
deals_joined.to_parquet(joined_path, index=False)
print(f'Wrote joined parquet: {joined_path} (rows: {len(deals_joined)})')


Joining companies onto deals (left join on CompanyID)
Before shape: (30680, 39), After shape: (30680, 79), Matched company rows: 30680
Wrote joined parquet: deals_joined.parquet (rows: 30680)


In [ ]:
# Building some simple features

for dtcol in ['DealDate','AnnouncedDate']:
    deals_joined[dtcol + '_year'] = deals_joined[dtcol].dt.year

deals_joined['Years_since_inceptions'] = deals_joined['DealDate_year'] - deals_joined['YearFounded']

deals_joined = deals_joined.sort_values(['CompanyID', 'DealNo'])
deals_joined['DealSize_prev'] = deals_joined.groupby('CompanyID')['DealSize'].shift(1)
deals_joined['PostValuation_prev'] = deals_joined.groupby('CompanyID')['PostValuation'].shift(1)
deals_joined.loc[deals_joined['DealNo'] == 1, ['DealSize_prev', 'PostValuation_prev']] = pd.NA

In [ ]:
deals_joined[deals_joined['CompanyID']=='100022-14'].head()[['CompanyID', 'DealNo', 'DealSize', 'DealSize_prev', 'PostValuation', 'PostValuation_prev', 'Years_since_inceptions']]

,CompanyID,DealNo,DealSize,DealSize_prev,PostValuation,PostValuation_prev,Years_since_inceptions
25191,100022-14,1,0.400000,NaN,NaN,NaN,0.0
24926,100022-14,2,0.050000,0.400000,NaN,NaN,1.0
23958,100022-14,3,1.175000,0.050000,21.174999,NaN,2.0
19779,100022-14,4,2.999999,1.175000,37.999999,21.174999,4.0
17480,100022-14,5,5.099998,2.999999,45.099998,37.999999,6.0


In [ ]:
# Imputing with Random Forest

spec = util.spec_from_file_location('imputer_mod', 'scripts/impute_postvaluation.py')
imputer = util.module_from_spec(spec)
spec.loader.exec_module(imputer)

dfimputed, model = imputer.impute_missing_targets_rf(
    deals_joined,
    numerical = [
        'PercentAcquired',
        'InvestorOwnership', 'TotalNewDebt',
        'ContingentPayout', 'Employees_x', 
        'Years_since_inceptions', 
        'Years_since_inceptions'
        # 'Employees_y', 'YearFounded', 'DealDate_year', 'AnnouncedDate_year'
        ],
    numerical_to_log=[
        'DealSize_prev', 'PostValuation_prev',
        'DealSize', 'PremoneyValuation', 'RaisedToDate', 'TotalInvestedCapital', 
        'PostValuation_prev', 'DealSize_prev',
        'DebtRaisedInRound', 'TotalInvestedEquity','TotalRaised','TotalRaisedNativeAmount'
        ],
    categorical=[
        'DealStatus','DealSizeStatus','PostValuationStatus','VCRound','VCRoundUp_Down_Flat',
        'DealType','DealType2','DealType3','DealClass','NativeCurrencyOfDeal','AddOn',
        'BusinessStatus_x','FinancingStatus','ExitScope','CompanyFinancingStatus',
        'TotalRaisedNativeCurrency','OwnershipStatus','Universe','Exchange','PrimaryContactPrefix',
        'PrimaryContactSuffix'
        ],
    target_col='PostValuation'
)


# Train RMSE: 598.9358, Train R2: 0.9569
# Test  RMSE: 764.9720, Test R2: 0.8869

In [47]:
# Imputing with Similarity

spec = util.spec_from_file_location('imputer_mod', 'scripts/impute_postvaluation.py')
imputer = util.module_from_spec(spec)
spec.loader.exec_module(imputer)

df_imputed = deals_joined.copy()

mask_missing = df_imputed['PostValuation'].isnull()
df_imputed.loc[mask_missing, 'PostValuation'] = df_imputed[mask_missing].apply(
    lambda row: imputer.impute_target_by_similarity(
        row,
        df_imputed,
        categorical_cols=['DealStatus'],
        numerical_cols=[
        'DealSize_prev', 'PostValuation_prev', 'Years_since_inceptions',
        'TotalNewDebt',
        'ContingentPayout',
        'DealDate_year',
        'DealSize', 'RaisedToDate',
        'DebtRaisedInRound','TotalRaised',
        ],
        range_dict={'DealSize': 0.80},
        target_col='PostValuation'
    ),
    axis=1
)

In [48]:
df_imputed[mask_missing][['CompanyID','PostValuation','DealSize','DealNo','VCRound']].head(5)

,CompanyID,PostValuation,DealSize,DealNo,VCRound
25191,100022-14,1.785714,0.400000,1,None
24926,100022-14,NaN,0.050000,2,None
6427,100022-14,70.962763,15.000000,7,5th Round
4775,100022-14,34.869020,3.973625,8,None
26010,100082-89,10.000000,2.860000,1,1st Round


# Imputation Summary

- Missing `PostValuation` values were imputed using RandomForest and similarity-based methods.
- Imputed values are stored in new columns
- Did feature selection, log-transforms, and categorical encoding 